# This is for wwall subroutine in vacuum_vac.f, completed

In [1]:
using Pkg
Pkg.activate("..")
Pkg.instantiate()
using JPEC, Plots

  Activating project at `~/Desktop/code/GPEC_hackaton/JPEC`


In [2]:
const pye = π ;
vac_math_path = joinpath(@__DIR__, "..","src", "Vacuum", "Vacuum_math.jl")
include(vac_math_path)

spl1d2! (generic function with 1 method)

In [3]:
"""
    bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                           xmin::Float64, xmax::Float64,
                           zmin::Float64, zmax::Float64)

Calculates the minimum and maximum X and Z coordinates within a specified range
of two input vectors. This function modifies the provided `Ref` arguments in-place.

# Arguments
- `x1::Vector{Float64}`: Input vector for X coordinates.
- `z1::Vector{Float64}`: Input vector for Z coordinates.
- `n1::Int`: Starting index for the range (Fortran 1-based index).
- `n2::Int`: Ending index for the range (Fortran 1-based index).
- `xmin::Float64`: Reference to store the calculated minimum X value.
- `xmax::Float64`: Reference to store the calculated maximum X value.
- `zmin::Float64`: Reference to store the calculated minimum Z value.
- `zmax::Float64`: Reference to store the calculated maximum Z value.
"""
function bounds!(x1::Vector{Float64}, z1::Vector{Float64}, n1::Int, n2::Int,
                           xmin::Float64, xmax::Float64,
                           zmin::Float64, zmax::Float64)

    # Check whether n1 is higher than n2
    if n1 > n2
        @warn "Starting index n1 ($n1) is greater than ending index n2 ($n2)."
        return nothing
    end
    if n1 < 1 || n2 > length(x1) || n2 > length(z1)
        error("Indices out of bounds.")
    end

    # 최소/최대 값 계산
    xmin = minimum(view(x1, n1:n2))
    xmax = maximum(view(x1, n1:n2))
    zmin = minimum(view(z1, n1:n2))
    zmax= maximum(view(z1, n1:n2))

end

bounds!

In [4]:
"""
    eqarcw(xin, zin, mw1)

This function performs arc length re-parameterization of a 2D curve. It takes an
input curve defined by `(xin, zin)` coordinates and re-samples it such that
the new points `(xout, zout)` are equally spaced in arc length.

# Arguments
- `xin::AbstractVector{Float64}`: Array of x-coordinates of the input curve.
- `zin::AbstractVector{Float64}`: Array of z-coordinates of the input curve.
- `mw1::Int`: Number of points in the input and output curves.

# Returns
- `xout::Vector{Float64}`: Array of x-coordinates of the arc-length re-parameterized curve.
- `zout::Vector{Float64}`: Array of z-coordinates of the arc-length re-parameterized curve.
- `ell::Vector{Float64}`: Array of cumulative arc lengths for the input curve.
- `thgr::Vector{Float64}`: Array of re-parameterized 'theta' values corresponding to equal arc lengths.
- `thlag::Vector{Float64}`: Array of normalized 'theta' values for the input curve (0 to 1).
"""
function eqarcw!(xin::Vector{Float64}, zin::Vector{Float64}, mw1::Int)
    thlag = zeros(Float64, mw1)
    ell = zeros(Float64, mw1)
    thgr = zeros(Float64, mw1)
    xout = zeros(Float64, mw1)
    zout = zeros(Float64, mw1)

    # Initialize thlag as normalized parameter from 0 to 1
    for iw in 1:mw1
        thlag[iw] = (1.0 / (mw1 - 1)) * (iw - 1)
    end

    # Calculate cumulative arc length
    ell[1] = 1.0e-8 # Small non-zero initial length
    for iw in 2:mw1
        thet = (thlag[iw] + thlag[iw - 1]) / 2.0
        
        # Get derivative of xin with respect to thlag
        lagrange1d!(thlag, xin, mw1, 3, thet, nothing , d_xin_d_thlag)
        lagrange1d!(thlag, zin, mw1, 3, thet, nothing, d_zin_d_thlag)
        
        # Calculate arc length segment
        xtzt = norm([d_xin_d_thlag, d_zin_d_thlag])
        ell[iw] = ell[iw - 1] + xtzt / (mw1 - 1)
    end

    # Re-parameterize based on equal arc lengths
    for i in 1:mw1
        elgr = (ell[mw1] / (mw1 - 1)) * (i - 1)
        # Interpolate thlag based on equal arc lengths
        thgr[i], _ = lag(ell, thlag, mw1, 3, elgr, 0)
    end

    # Get xout and zout using the re-parameterized thgr
    for i in 1:mw1
        ttt = thgr[i]
        # Interpolate xin, zin at ttt
        lagrange1d!(thlag, xin, mw1, 3, ttt, xout[i], nothing)
        lagrange1d(thlag, zin, mw1, 3, ttt, zout[i], nothing)
    end

    return xout, zout, ell, thgr, thlag
end

eqarcw!

### adjustb

In [5]:
"""
    adjustb!(betin, betout_ref, a_, bw_, cw_, dw_, xmaj_, plrad_, ishape_)

Adjusts the `betin` angle based on the `ishape_` and other wall/plasma parameters.
This function takes `betout_ref` as a `Ref` so it can modify the output value in-place.

# Arguments
- `betin::Float64`: Input angle.
- `betout_ref::Ref{Float64}`: Reference to the output angle, which will be modified.
- `a_::Float64`: Wall parameter.
- `bw_::Float64`: Wall parameter (elongation or height).
- `cw_::Float64`: Wall parameter (center or offset).
- `dw_::Float64`: Wall parameter (triangularity).
- `xmaj_::Float64`: Magnetic axis X coordinate.
- `plrad_::Float64`: Plasma radius.
- `ishape_::Int`: Integer indicating the wall shape type.
"""
function adjustb!(betin::Float64, betout_ref::Float64, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)

    local r0::Float64 = 0.0 # Initialize to avoid UndefVarError if ishape_ is not 21 or 31
    local r::Float64 = 0.0  # Initialize

    if ishape_ == 31
        r0 = cw_
        r  = a_
    elseif ishape_ == 21 # Use elseif for mutually exclusive conditions
        r0 = xmaj_ + cw_ * plrad_
        r  = plrad_ * (1.0 + a_ - cw_)
    else
        @warn "adjustb!: Unsupported ishape_ value: $ishape_. r0 and r will remain 0.0."
    end

    bet2 = betin
    
    # Check for potential division by zero if tan(bet2) is infinite or if bw_ is zero.
    # atan(Inf) is handled correctly by Julia, but tan(bet2) could be very large
    # if bet2 is close to pi/2 + n*pi. If bw_ is zero, division by zero occurs.
    if bw_ == 0.0
        @warn "adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN."
        betout_ref[] = NaN
        return nothing
    end

    betout_ref[] = abs(atan(tan(bet2) / bw_))

    return nothing # Fortran's `return` from a subroutine
end

adjustb!

### Test adjustb

In [6]:
"""
    adjustb!(betin, betout_ref, a_, bw_, cw_, dw_, xmaj_, plrad_, ishape_)

Adjusts the `betin` angle based on the `ishape_` and other wall/plasma parameters.
This function takes `betout_ref` as a `Ref` so it can modify the output value in-place.

# Arguments
- `betin::Float64`: Input angle.
- `betout_ref::Ref{Float64}`: Reference to the output angle, which will be modified.
- `a_::Float64`: Wall parameter.
- `bw_::Float64`: Wall parameter (elongation or height).
- `cw_::Float64`: Wall parameter (center or offset).
- `dw_::Float64`: Wall parameter (triangularity).
- `xmaj_::Float64`: Magnetic axis X coordinate.
- `plrad_::Float64`: Plasma radius.
- `ishape_::Int`: Integer indicating the wall shape type.
"""
function adjustb!(betin::Float64, betout_ref::Ref{Float64}, a_::Float64, bw_::Float64, cw_::Float64, dw_::Float64,
                 xmaj_::Float64, plrad_::Float64, ishape_::Int)

    # These local variables r0 and r are correctly scoped and used for intermediate calculations.
    local r0::Float64 = 0.0
    local r::Float64 = 0.0

    if ishape_ == 31
        r0 = cw_
        r  = a_
    elseif ishape_ == 21
        r0 = xmaj_ + cw_ * plrad_
        r  = plrad_ * (1.0 + a_ - cw_)
    else
        @warn "adjustb!: Unsupported ishape_ value: $ishape_. r0 and r will remain 0.0."
    end

    bet2 = betin # No change here, bet2 is a local copy of betin

    if bw_ == 0.0
        @warn "adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN."
        betout_ref[] = NaN # Correctly assign NaN to the value inside the Ref
        return nothing
    end
    # The line below was causing the error because `betout_ref` was a Float64, not a Ref
    # It should assign to the *value inside* the Ref, using `betout_ref[] = ...`
    betout_ref[] = abs(atan(tan(bet2) / bw_)) # Correctly assign to the value inside the Ref
    return nothing # Explicitly return nothing as the primary effect is the side effect on betout_ref
end

# --- Direct function execution and result verification ---

println("--- Starting direct execution tests for adjustb! ---")

# Common test parameters (these don't affect the core calculation of betout_ref in this function's logic)
a_val = 1.0
cw_val = 0.5
dw_val = 0.1
xmaj_val = 1.6
plrad_val = 0.5

# Test Case 1: Basic operation (bw_ = 1.0)
betin_val_1 = pi / 4 # 45 degrees
bw_val_1 = 1.0
betout_ref_1 = Ref(0.0) # Initialize as Ref{Float64}
adjustb!(betin_val_1, betout_ref_1, a_val, bw_val_1, cw_val, dw_val, xmaj_val, plrad_val, 21) # ishape does not affect calculation
expected_1 = abs(atan(tan(betin_val_1) / bw_val_1))
println("\n--- Test Case 1: Basic Operation (betin = pi/4, bw_ = 1.0) ---")
println("Input betin: $(betin_val_1) (rad), tan(betin): $(tan(betin_val_1))")
println("Input bw_: $(bw_val_1)")
println("Calculated betout: $(betout_ref_1[])") # Access the value inside the Ref
println("Expected betout: $(expected_1)")
if abs(betout_ref_1[] - expected_1) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end

# Test Case 2: BW_ scaling effect
betin_val_2 = pi / 4
bw_val_2 = 0.5
betout_ref_2 = Ref(0.0) # Initialize as Ref{Float64}
adjustb!(betin_val_2, betout_ref_2, a_val, bw_val_2, cw_val, dw_val, xmaj_val, plrad_val, 21)
expected_2 = abs(atan(tan(betin_val_2) / bw_val_2))
println("\n--- Test Case 2: BW_ Scaling (betin = pi/4, bw_ = 0.5) ---")
println("Input betin: $(betin_val_2) (rad), tan(betin): $(tan(betin_val_2))")
println("Input bw_: $(bw_val_2)")
println("Calculated betout: $(betout_ref_2[])") # Access the value inside the Ref
println("Expected betout: $(expected_2)")
if abs(betout_ref_2[] - expected_2) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end

# Test Case 3: Negative betin
betin_val_3 = -pi / 4
bw_val_3 = 1.0
betout_ref_3 = Ref(0.0) # Initialize as Ref{Float64}
adjustb!(betin_val_3, betout_ref_3, a_val, bw_val_3, cw_val, dw_val, xmaj_val, plrad_val, 21)
expected_3 = abs(atan(tan(betin_val_3) / bw_val_3))
println("\n--- Test Case 3: Negative betin (betin = -pi/4, bw_ = 1.0) ---")
println("Input betin: $(betin_val_3) (rad), tan(betin): $(tan(betin_val_3))")
println("Input bw_: $(bw_val_3)")
println("Calculated betout: $(betout_ref_3[])") # Access the value inside the Ref
println("Expected betout: $(expected_3)")
if abs(betout_ref_3[] - expected_3) < 1e-9
    println("Result matches: ✅")
else
    println("Result mismatch: ❌")
end

# Test Case 4: bw_ = 0.0 (Error handling)
betin_val_4 = pi / 4
bw_val_4 = 0.0
betout_ref_4 = Ref(0.0) # Initialize as Ref{Float64}
println("\n--- Test Case 4: bw_ = 0.0 (Expected Warning and NaN) ---")
adjustb!(betin_val_4, betout_ref_4, a_val, bw_val_4, cw_val, dw_val, xmaj_val, plrad_val, 21)
println("Input betin: $(betin_val_4) (rad)")
println("Input bw_: $(bw_val_4)")
println("Calculated betout: $(betout_ref_4[])") # Access the value inside the Ref
if isnan(betout_ref_4[])
    println("Result is NaN (as expected): ✅")
else
    println("Result is not NaN (Error): ❌")
end

# Test Case 5: Unsupported ishape_
betin_val_5 = pi / 6
bw_val_5 = 1.0
betout_ref_5 = Ref(0.0) # Initialize as Ref{Float64}
ishape_val_5 = 99 # Unsupported value
println("\n--- Test Case 5: Unsupported ishape_ (Expected Warning) ---")
adjustb!(betin_val_5, betout_ref_5, a_val, bw_val_5, cw_val, dw_val, xmaj_val, plrad_val, ishape_val_5)
expected_5 = abs(atan(tan(betin_val_5) / bw_val_5))
println("Input ishape_: $(ishape_val_5)")
println("Calculated betout: $(betout_ref_5[])") # Access the value inside the Ref
println("Expected betout (calculation formula is the same): $(expected_5)")
if abs(betout_ref_5[] - expected_5) < 1e-9
    println("Result matches (only warning issued): ✅")
else
    println("Result mismatch: ❌")
end

println("\n--- Direct execution tests for adjustb! completed ---")

--- Starting direct execution tests for adjustb! ---

--- Test Case 1: Basic Operation (betin = pi/4, bw_ = 1.0) ---
Input betin: 0.7853981633974483 (rad), tan(betin): 0.9999999999999999
Input bw_: 1.0
Calculated betout: 0.7853981633974483
Expected betout: 0.7853981633974483
Result matches: ✅

--- Test Case 2: BW_ Scaling (betin = pi/4, bw_ = 0.5) ---
Input betin: 0.7853981633974483 (rad), tan(betin): 0.9999999999999999
Input bw_: 0.5
Calculated betout: 1.1071487177940904
Expected betout: 1.1071487177940904
Result matches: ✅

--- Test Case 3: Negative betin (betin = -pi/4, bw_ = 1.0) ---
Input betin: -0.7853981633974483 (rad), tan(betin): -0.9999999999999999
Input bw_: 1.0
Calculated betout: 0.7853981633974483
Expected betout: 0.7853981633974483
Result matches: ✅

--- Test Case 4: bw_ = 0.0 (Expected Warning and NaN) ---


┌ Warning: adjustb!: Division by zero detected (bw_ is 0.0). Setting betout_ref to NaN.
└ @ Main In[6]:38


Input betin: 0.7853981633974483 (rad)
Input bw_: 0.0
Calculated betout: NaN
Result is NaN (as expected): ✅

--- Test Case 5: Unsupported ishape_ (Expected Warning) ---
Input ishape_: 99
Calculated betout: 0.5235987755982988
Expected betout (calculation formula is the same): 0.5235987755982988
Result matches (only warning issued): ✅

--- Direct execution tests for adjustb! completed ---


┌ Warning: adjustb!: Unsupported ishape_ value: 99. r0 and r will remain 0.0.
└ @ Main In[6]:32


### d3dvesl, d3dwall

In [7]:
"""
    d3dvesl!(r0, z0, a0, e0, ar, az, nval, zst, r, z, npts, ier_ref)

Defines the shape of the DIII-D vacuum vessel based on Fourier series coefficients.

# Arguments
- `r0::Float64`: Reference major radius.
- `z0::Float64`: Reference vertical position.
- `a0::Float64`: Minor radius scaling factor.
- `e0::Float64`: Elongation scaling factor.
- `ar::Vector{Float64}`: Fourier coefficients for radial component.
- `az::Vector{Float64}`: Fourier coefficients for vertical component.
- `nval::Int`: Number of Fourier coefficients.
- `zst::Float64`: Starting vertical position for calculation (used in an unused branch).
- `r::Vector{Float64}`: Output vector for R coordinates of the wall (modified in-place).
- `z::Vector{Float64}`: Output vector for Z coordinates of the wall (modified in-place).
- `npts::Int`: Number of points to generate for the wall.
- `ier_ref::Ref{Int}`: Error flag (0 for success, 1 for error) passed by reference.
"""
function d3dvesl!(r0::Float64, z0::Float64, a0::Float64, e0::Float64,
                 ar::Vector{Float64}, az::Vector{Float64}, nval::Int, zst::Float64,
                 r::Vector{Float64}, z::Vector{Float64}, npts::Int, ier_ref::Int)

    pii = acos(-1.0) # Using Julia's built-in pi constant from Pye
    ier_ref = 0 # Initialize error flag

    local arcst::Float64

    if abs(z0 - zst) <= 1.0e-6
        # This branch is usually taken, as zstart is set to 0.0 in d3dwall
        arcst = 0.0
    else
        local isgn::Int
        local zpcmp::Float64
        local arci::Float64
        local arcf::Float64
        local dfi::Float64
        local arca::Float64
        local zza::Float64
        local arcb::Float64
        local zzb::Float64
        local ackb::Float64
        local arcc::Float64
        local zzc::Float64
        local ackc::Float64
        local dzp::Float64
        local dzm::Float64
        local dzt::Float64
        local dcf1::Float64
        local dcf2::Float64
        local zdf::Float64

        if z0 < zst
            isgn = +1
        end
        if z0 > zst
            isgn = -1
        end
        zpcmp = (zst - z0) / (a0 * e0)
        arci = 0.0
        arcf = 0.5 * pii
        dfi = (arcf - arci) / npts
        arca = arci
        zza = 0.0
        for j = 2:npts
            arcb = arca + isgn * dfi
            zzb = 0.0
            for k = 1:nval
                ackb = k * arcb
                zzb = zzb + az[k] * sin(ackb)
            end # 5 continue
            if (zza - zpcmp) * (zzb - zpcmp) <= 0.0
                arcc = arcb + isgn * dfi
                zzc = 0.0
                for k = 1:nval
                    ackc = k * arcc
                    zzc = zzc + az[k] * sin(ackc)
                end # 10 continue
                @goto label_25
            else
                arca = arcb
                zza = zzb
            end
        end # 20 continue
        ier_ref[] = 1
        return
        @label label_25 # 25 continue
        dzp = zzc - zzb
        dzm = zzb - zza
        dzt = zzc - zza
        dcf1 = dfi * (dzm / dzp + dzp / dzm) / dzt
        dcf2 = dfi * (1.0 / dzp - 1.0 / dzm) / dzt
        zdf = zpcmp - zzb
        arcst = arcb + dcf1 * zdf + dcf2 * zdf * zdf
    end

    arc0 = arcst
    arc1 = arcst + 2.0 * pii
    darc = (arc1 - arc0) / npts

    for j = 1:npts
        arc = arc0 + (j - 1.0) * darc
        sumr = 0.0
        sumz = 0.0
        for k = 1:nval
            arck = k * arc
            sumr = sumr + ar[k] * cos(arck)
            sumz = sumz + az[k] * sin(arck)
        end # 50 continue
        rpval = r0 + a0 * sumr
        zpval = z0 - e0 * a0 * sumz
        r[j] = rpval
        z[j] = zpval
    end # 100 continue

    return nothing
end

d3dvesl!

In [8]:
using Printf # For @printf

"""
    d3dwall!(xwall, ywall, mthh, iomod, iotty1, rext_in, zlim_val_in)

Defines the shape of the DIII-D wall by calling `d3dvesl`.

# Arguments
- `xwall::Vector{Float64}`: Output vector for X coordinates of the DIII-D wall (modified in-place).
- `ywall::Vector{Float64}`: Output vector for Y (Z in Cartesian) coordinates of the DIII-D wall (modified in-place).
- `mthh::Int`: Number of points to generate for the wall.
- `iomod::IO`: Output stream for general messages.
- `iotty1::IO`: Output stream for terminal messages.
- `rext_in::Float64`: External scaling factor for the wall.
- `zlim_val_in::Float64`: Global z-limit value (used in `d3dvesl` implicitly).
"""
function d3dwall!(xwall::Vector{Float64}, ywall::Vector{Float64}, mthh::Int,
                 iomod::IO, iotty1::IO, rext_in::Float64, zlim_val_in::Float64)

    ncdf = 26
    rwi = [
        1.0, 0.05526794, -0.1738114, 0.01850757, 0.03714965, -0.02882647,
        -0.002357329, 0.009548103, -0.01214923, -0.001853416, 0.006837493,
        -0.001711245, 0.002270762, 0.003689963, -0.003959393, -0.001098017,
        0.003745465, -0.0002157904, -0.0003977743, -0.0002725623,
        -0.001005857, -0.000004579016, 0.002396789, -0.0007057043,
        0.001158347, 0.0003552319
    ]
    zwi = [
        1.0, -0.03236632, -0.1629422, 0.06013983, 0.01167756, -0.02579542,
        0.01626464, -0.002085857, -0.009098639, 0.01022163, -0.004388253,
        -0.009367258, 0.008308497, 0.004765150, -0.004611675, -0.001121423,
        -0.0002501100, 0.4282634e-03, 0.2669702e-02, -0.1073800e-02, # Corrected: -0.1073800d-02 from Fortran
        -0.2191338e-02, 0.1328267e-02, 0.5050959e-03, -0.5758863e-03,
        0.9348883e-03, 0.7094351e-03
    ]

    rwall0 = 1.6400000
    zwall0 = 0.0000000
    awall0 = 0.8839410
    ewall0 = 1.4037020

    nwcoef = ncdf

    rwll = rwall0
    zwll = zwall0
    awll = awall0 * rext_in # Use rext_in here
    ewll = ewall0

    # In the Fortran code, `zstart = zlim` and then `zlim = 0.0`, followed by `zstart = 0.0`.
    # This effectively makes `zstart` 0.0 for the call to `d3dvesl`.
    zstart = 0.0 # This overwrites zlim_val_in for the purpose of d3dvesl.

    nwalp = mthh

    ier_ref = 0 # Julia's way to pass an integer by reference for modification
    d3dvesl!(rwll, zwll, awll, ewll, rwi, zwi, nwcoef, zstart, xwall, ywall, nwalp, ier_ref)
    ier = ier_ref[] # Get the value from the reference

    @printf(iomod, "ier in d3dwall = %3d\n", ier)
    @printf(iotty1, "ier in d3dwall = %3d\n", ier)

    xwall[mthh+1] = xwall[1]
    ywall[mthh+1] = ywall[1]
end

d3dwall!

### Test d3dwall, d3dvesl

In [9]:

# 1. Define input parameters
# mthh is the number of points for the wall, also equivalent to npts for d3dvesl!
mthh_val = 200 # A reasonable number of points for a smooth curve
rext_val = 1.0 # External scaling factor, usually 1.0 for standard DIII-D wall
zlim_val = 0.0 # This value is overwritten to 0.0 inside d3dwall! for zstart

# 2. Initialize output vectors
# xwall and ywall need to be mthh_val + 1 long because d3dwall! adds the first point at the end
xwall_output = Vector{Float64}(undef, mthh_val + 1)
ywall_output = Vector{Float64}(undef, mthh_val + 1)

# 3. Set up IO streams
# `devnull` suppresses output. Use `stdout` to see messages in the console.
iomod_stream = devnull # You can change this to stdout if you want to see "ier in d3dwall" for iomod
iotty1_stream = stdout # Prints "ier in d3dwall" to your terminal

# 4. Call the d3dwall! function
println("Calling d3dwall! with mthh_val = $mthh_val and rext_val = $rext_val...")
d3dwall!(xwall_output, ywall_output, mthh_val, iomod_stream, iotty1_stream, rext_val, zlim_val)
println("d3dwall! call completed. Check output below:")

# 5. Verify the results
println("\n--- Verification ---")

# Check if the output vectors are populated
@printf "Is xwall_output empty? %s\n" isempty(xwall_output) ? "Yes" : "No"
@printf "Is ywall_output empty? %s\n" isempty(ywall_output) ? "Yes" : "No"

# Check the lengths
@printf "Length of xwall_output: %d (Expected: %d)\n" length(xwall_output) (mthh_val + 1)
@printf "Length of ywall_output: %d (Expected: %d)\n" length(ywall_output) (mthh_val + 1)

# Check if the first and last points match (as d3dwall! explicitly sets them)
# Use ≈ for approximate equality with floating-point numbers
@printf "xwall_output[1] (%f) vs xwall_output[end] (%f): Match? %s\n" xwall_output[1] xwall_output[end] (xwall_output[1] ≈ xwall_output[end]) ? "Yes" : "No"
@printf "ywall_output[1] (%f) vs ywall_output[end] (%f): Match? %s\n" ywall_output[1] ywall_output[end] (ywall_output[1] ≈ ywall_output[end]) ? "Yes" : "No"

# Print a few sample points to visually inspect the shape (optional, for debugging)
println("\n--- Sample Points (R, Z) ---")
for i in 1:min(5, mthh_val)
    @printf "Point %3d: R = %8.4f, Z = %8.4f\n" i xwall_output[i] ywall_output[i]
end
println("...")
for i in max(1, mthh_val-4):mthh_val+1
    @printf "Point %3d: R = %8.4f, Z = %8.4f\n" i xwall_output[i] ywall_output[i]
end

using Plots
plot(xwall_output, ywall_output, seriestype=:scatter, legend=false, aspect_ratio=:equal,
     xlabel="R (m)", ylabel="Z (m)", title="DIII-D Vacuum Vessel Shape")
savefig("d3d_vessel_shape.png") # Saves the plot to a file
println("\nPlot saved to d3d_vessel_shape.png")

Calling d3dwall! with mthh_val = 200 and rext_val = 1.0...
ier in d3dwall =   0
d3dwall! call completed. Check output below:

--- Verification ---
Is xwall_output empty? No
Is ywall_output empty? No
Length of xwall_output: 201 (Expected: 201)
Length of ywall_output: 201 (Expected: 201)
xwall_output[1] (2.446648) vs xwall_output[end] (2.446648): Match? Yes
ywall_output[1] (0.000000) vs ywall_output[end] (0.000000): Match? Yes

--- Sample Points (R, Z) ---
Point   1: R =   2.4466, Z =   0.0000
Point   2: R =   2.4459, Z =  -0.0268
Point   3: R =   2.4440, Z =  -0.0528
Point   4: R =   2.4418, Z =  -0.0778
Point   5: R =   2.4405, Z =  -0.1022
...
Point 196: R =   2.4405, Z =   0.1269
Point 197: R =   2.4405, Z =   0.1022
Point 198: R =   2.4418, Z =   0.0778
Point 199: R =   2.4440, Z =   0.0528
Point 200: R =   2.4459, Z =   0.0268
Point 201: R =   2.4466, Z =   0.0000

Plot saved to d3d_vessel_shape.png


### wwall

In [10]:

# Constants and initial values (replace with actual values from your setup)
const nths = 100 # Example value
const pye = π
global aw = 5.0 # Will be saved and restored
global bw = 0.0 # Will be saved and restored
global a = 3.0
global b = 2.0
global cw = 1.0
global dw = 0.0
global tw = 0.0
global abulg = 0.0
global bbulg = 0.0
global tbulg = 0.0
global ishape = 0
global farwal = false
global leqarcw = 0

# Placeholder for arrays that would be in vglobal_mod
global xinf = zeros(Float64, nths) # Assuming size nths based on context
global zinf = zeros(Float64, nths) # Assuming size nths based on context
global xma = 0.0
global zma = 0.0

global xmx = 0.0 # Max x-coordinate, modified and potentially used elsewhere
global zmx = 0.0 # Max z-coordinate, seems to be a typo/unused, zma is used instead
global iplt = 0 # Integer flag, modified to 1 at the end of the function if <= 0

global npots0::Int
global npots::Int
global lfix::Bool
global insect::Bool
global csmin::Float64
global thetatmp::Vector{Float64}
global xwaltmp::Vector{Float64}
global zwaltmp::Vector{Float64}
global xpptmp::Vector{Float64}
global ww1tmp::Vector{Float64}
global ww2tmp::Vector{Float64}
global ww3tmp::Vector{Float64}
global tabtmp::Vector{Float64}
global rioptmp::Vector{Int}

# Variables for common calculations
global awsave::Float64
global bwsave::Float64
global isph::Int
global xmnp::Float64 = 0.0 # Use Ref for single value output from `bounds!`
global xmxp::Float64 = 0.0
global zmnp::Float64 = 0.0
global zmxp::Float64 = 0.0
global xmin::Float64 = 0.0
global xmax::Float64 = 0.0
global zmin::Float64 = 0.0
global zmax::Float64 = 0.0
global plrad::Float64
global xmaj::Float64
global zmid::Float64
global zrad::Float64
global scale::Float64
global delta1::Float64
global mthalf::Int
global inside::Int = 0 # Initialize `inside` as it's incremented



"""
    wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})

This subroutine calculates the coordinates of the wall (xwal1, zwal1) based on
various geometric shapes and plasma parameters. It modifies `xwal1` and `zwal1` in place.

# Arguments
- `nqnqnq::Int`: An integer parameter (its specific use isn't clear from the snippet,
                  but it's passed as an argument).
- `xwal1::Vector{Float64}`: Array to store the calculated x-coordinates of the wall.
                            Modified in-place.
- `zwal1::Vector{Float64}`: Array to store the calculated z-coordinates of the wall.
                            Modified in-place.

c-----------------------------------------------------------------------
c     ishape< 0 Spherical topology
c     ishape<10 Closed toroidal topology
c     ishape<20 Solid conductors not linking plasma
c     ishape<30 Toroidal conductor with a toroidally symmetric gap.
c               Geometry correlated to plasma position and geometry.
c     ishape<40 Toroidal conductor with a toroidally symmetric gap.
c               Geometry independent of plasma.
c-----------------------------------------------------------------------

"""
function wwall!(nqnqnq::Int, xwal1::Vector{Float64}, zwal1::Vector{Float64})

    global aw, bw, a, b, cw, dw, tw, abulg, bbulg, tbulg, ishape, farwal, leqarcw
    global xinf, zinf, xma, zma, xmx, zmx, iplt
    global npots0, npots, lfix, insect, csmin, thetatmp, xwaltmp, zwaltmp, xpptmp, ww1tmp, ww2tmp, ww3tmp, tabtmp, rioptmp
    global awsave, bwsave, isph, xmnp, xmxp, zmnp, zmxp, xmin, xmax, zmin, zmax, plrad, xmaj, zmid, zrad, scale, delta1, mthalf, inside
    global xpp_out, zpp_out # Declare for eqarcw! if they are globals

    iop = Vector{Int}(undef, 2) 
    xpp = zeros(Float64, nths)
    zpp = zeros(Float64, nths)
    ww1 = zeros(Float64, nths)
    ww2 = zeros(Float64, nths)
    ww3 = zeros(Float64, nths)
    thet = zeros(Float64, nths)
    tabx = zeros(Float64, 3)
    tabz = zeros(Float64, 3)

    # Fortran DATA statement
    iplt = 0


    # global aw = 5.0 # Will be saved and restored
    # global bw = 0.0 # Will be saved and restored
    # global a = 3.0
    # global b = 2.0
    # global cw = 1.0
    # global dw = 0.0
    # global tw = 0.0
    # global abulg = 0.0
    # global bbulg = 0.0
    # global tbulg = 0.0
    # global ishape = 0
    # global farwal = false
    # global leqarcw = 0
    
    # # Placeholder for arrays that would be in vglobal_mod
    # global xinf = zeros(Float64, nths) # Assuming size nths based on context
    # global zinf = zeros(Float64, nths) # Assuming size nths based on context
    # global xma = 0.0
    # global zma = 0.0
    
    # global xmx = 0.0 # Max x-coordinate, modified and potentially used elsewhere
    # global zmx = 0.0 # Max z-coordinate, seems to be a typo/unused, zma is used instead
    # global iplt = 0 # Integer flag, modified to 1 at the end of the function if <= 0
    
    # global npots0::Int
    # global npots::Int
    # global lfix::Bool
    # global insect::Bool
    # global csmin::Float64
    # global thetatmp::Vector{Float64}
    # global xwaltmp::Vector{Float64}
    # global zwaltmp::Vector{Float64}
    # global xpptmp::Vector{Float64}
    # global ww1tmp::Vector{Float64}
    # global ww2tmp::Vector{Float64}
    # global ww3tmp::Vector{Float64}
    # global tabtmp::Vector{Float64}
    # global rioptmp::Vector{Int}
    
    # # Variables for common calculations
    # global awsave::Float64
    # global bwsave::Float64
    # global isph::Int
    # global xmnp::Float64 = 0.0 # Use Ref for single value output from `bounds!`
    # global xmxp::Float64 = 0.0
    # global zmnp::Float64 = 0.0
    # global zmxp::Float64 = 0.0
    # global xmin::Float64
    # global xmax::Float64 = 0.0
    # global zmin::Float64
    # global zmax::Float64
    # global plrad::Float64
    # global xmaj::Float64
    # global zmid::Float64
    # global zrad::Float64
    # global scale::Float64
    # global delta1::Float64
    # global mthalf::Int
    # global inside::Int = 0 # Initialize `inside` as it's incremented



    # local aw, bw, ishape, xmx, zma, iplt,xmin, xmax, zmin, zmax


    # Constants calculated based on nths
    mth = nths # Assuming nths is equivalent to mth for indexing purposes in this context
    mth1 = nths # mth1 = mth + 1 in Fortran
    mth2 = nths # mth2 = mth + 2 in Fortran
    dth = 2.0 * pye / (mth2 - 1) # (2.0*pi / (mth+1))

    # --- Computations ---
    xpp[1] = 1.0
    zpp[1] = 1.0
    ww3[1] = 1.0
    tabx[1] = 1.0
    tabz[1] = 1.0

    awsave = aw
    bwsave = bw
    insect = false
    isph = 0

    # --- Shape Logic ---
    if a < -100.0
        isph = 1
        ishape = -10
        bounds!(xinf, zinf, 1, mth, xmnp, xmxp, zmnp, zmxp)
        xmin = xmnp
        xmax = xmxp
        zmin = zmnp
        zmax = zmxp
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        zmid = 0.5 * (zmax + zmin)
        hrad = xmax + aw * (xmax - xmaj)
        vrad = zmax + bw * (zmax - zmid)
        for i in 1:mth1
            xi = xinf[i] - xmaj
            zeta = zinf[i] - zmid
            bbb = (xi * vrad)^2 + (zeta * hrad)^2
            ccc = -xmaj * vrad * xi + hrad * sqrt(bbb - (zeta * xmaj)^2)
            xwal1[i] = xmaj + xi * vrad * ccc / bbb
            zwal1[i] = zmid + zeta * vrad * ccc / bbb
        end
        @goto cleanup # Direct jump to cleanup section
    elseif a > -10.0
        lfix = true
    else
        lfix = false # Initialize lfix for all paths
    end

    if farwal
        @info "No wall"
        return
    end

    xshift = a
    mthalf = floor(Int, mth2 / 2) 
    bounds!(xinf, zinf, 1, mth, xmin, xmax, zmin, zmax)
    plrad = 0.5 * (xmax - xmin)
    xmaj = 0.5 * (xmax + xmin)
    zmid = 0.5 * (zmax + zmin)
    zrad = 0.5 * (zmax - zmin)
    scale = (zmax - zmin)

    if ((xmax - xmin) / 2.0) > scale
        scale = (xmax - xmin) / 2.0
    end

    scale = 1.0
    aw = aw * scale
    bw = bw * scale
    delta1 = dw * (xinf[1] - xma)

    # ishape=2 Elliptical shell
    if ishape == 2
        zh = sqrt(abs(zrad^2 - plrad^2))
        zah = a / zh
        zph = plrad / zh
        zmup = 0.5 * log((zrad + plrad) / (zrad - plrad)) 
        zmuw = log(zah + sqrt(zah^2 + 1)) 
        zxmup = exp(zmup)
        zxmuw = exp(zmuw)
        zbwal = zh * cosh(zmuw) # Major radius of wall
        bw = zbwal / a          # Elongation of wall
        for i in 1:mth2
            the = (i - 1) * dth
            xwal1[i] = xmaj + a * cos(the)
            zwal1[i] = -bw * a * sin(the)
        end
    end

    # ishape=3
    if ishape == 3
        for i in 1:mth2
            rr = (xinf[i] - xma)^2 + (zinf[i] - zma)^2
            ro = sqrt(rr)
            the = atan((zinf[i] - zma), (xinf[i] - xma)) # Use atan2 for correct quadrant
            thex = abs(the)
            lsgn = 1
            if xma > xinf[i]
                the = the + pye
            end
            if i <= mthalf
                if xma > xinf[i]
                    thex = pye - thex
                end
                thet[i] = abs(thex)
            end
            if !lfix
                ro = ro + delta1
                xwal1[i] = xma + lsgn * ro * cos(the)
                zwal1[i] = zma + lsgn * ro * sin(the)
            else
                xshift = (xmax + xmin) / 2.0
                xshift = a
                the = (i - 1) * dth
                xwal1[i] = xshift + aw * cos(the + dw * sin(the))
                zwal1[i] = zma - bw * sin(the)
            end
            if i > mthalf
                continue
            end
            if zwal1[i] < zmin
                continue
            end
            j = i
            insect = false
            jsmall = j
            jlarge = j
            ics = 1
            if xma >= xinf[i]
                ics = -1
            end
            while true
                if zinf[j] >= zwal1[i]
                    jsmall = j
                else
                    break
                end
                if j >= mthalf
                    continue # Fortran's cycle
                end
                if j < 1
                    continue # Fortran's cycle
                end
                j = j + ics
            end
            jlarge = j
            if abs(xinf[jsmall] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if abs(xinf[jlarge] - xma) >= abs(xwal1[i] - xma)
                insect = true
            end
            if !insect
                continue
            end
            inside += 1
        end
    end

    # ishape=4 Modified dee-shaped wall independent of plasma geometry
    if ishape == 4
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = cw + a * cos(the + dw * sin(the))
            zwal1[i] = -bw * a * sin(the + tw * sn2th) - aw * sn2th
        end
    end

    # ishape=5 Dee-shaped wall scaled to plasma
    if ishape == 5
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            sn2th = sin(2.0 * the)
            xwal1[i] = xmaj + cw * plrad + plrad * (1.0 + a - cw) * cos(the + dw * sin(the))
            zwal1[i] = -bw * plrad * (1.0 + a - cw) * sin(the + tw * sn2th) - aw * plrad * sn2th
        end
    end

    # ishape=6 Conforming shell
    if ishape == 6
        wcentr = xmaj
        # Fortran's minval(xinf(2:mth1))
        # Note: If xinf is guaranteed to have at least 2 elements, this is safe.
        csmin = min(0.1, 1e-1 * minimum(view(xinf, 2:mth1)))
        for i in 2:mth1
            alph = atan(xinf[i+1] - xinf[i-1], zinf[i-1] - zinf[i+1]) # Fortran's ATAN2
            xwal1[i] = xinf[i] + a * plrad * cos(alph)
            zwal1[i] = zinf[i] + a * plrad * sin(alph)
            # if the wall crosses the R=0 axis, force a thin center stack
            if xwal1[i] <= csmin
                xwal1[i] = csmin
            end
        end
        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    # ishape=7 Enclosing bean-shaped wall
    if ishape == 7
        cwr = cw * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = aw * (1.0 + bw * cos(the))
            the2 = cwr * sin(the)
            xofsw = xmax + a * plrad - aw * (1.0 + bw)
            xwal1[i] = xofsw + rho * cos(the2)
            zwal1[i] = -b * rho * sin(the2)
        end
    end

    # ishape=8 Wall of DIII-D
    if ishape == 8
        d3dwall!(xwal1, zwal1, mth, devnull, stdout, 1.0, 0.0)
    end

    # ishape=11 Dee-shaped conductor
    if ishape == 11
        for i in 1:mth2
            the = (i - 1) * dth
            plrad = 0.5 * (xmax - xmin)
            xwal1[i] = xmax + plrad * (a + aw - aw * cos(the + dw * sin(the)))
            zwal1[i] = -plrad * bw * sin(the)
        end
    end

    # ishape=12 Solid bean-shaped conductor on right
    if ishape == 12
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 - aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=13 Solid bean-shaped conductor on left
    if ishape == 13
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            rho = a0 + aw * plrad * cos(the)
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2)
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=21 Shell scaled to plasma. Gap on the inner side.
    if ishape == 21
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        a0b = (a0 + plrad * aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = xmaj + cw * plrad

        # Create Ref for adjustb! output
        blgrado_ref = 0.0
        blgradi_ref = 0.0

        # Call adjustb!
        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw * plrad / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * plrad * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * plrad * fermb
            xwal1[i] = xmaj + cw * plrad + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=24 Shell scaled to plasma. Gap on the outer side.
    if ishape == 24
        plrad = 0.5 * (xmax - xmin)
        xmaj = 0.5 * (xmax + xmin)
        a0 = plrad * (1.0 + aw - cw + a)
        brad = b * pye / 180.0
        wcentr = xmaj + cw * plrad
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * plrad * ferm
            the2 = brad * sin(the)
            xwal1[i] = xmaj + cw * plrad - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=31 Shell independent of plasma. Gap on the inner side.
    if ishape == 31
        a0 = a + aw
        a0b = (a0 + aw) * bw
        brad0 = b * pye / 180.0
        brad = brad0
        blgrad0 = bbulg * pye / 180.0
        wcentr = cw

        blgrado_ref = 0.0
        blgradi_ref = 0.0

        adjustb!(blgrad0, blgrado_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgrado = blgrado_ref[]

        dthb = (2.0 * aw / a0b) * (1.0 - sin(blgrado)) / cos(blgrado)
        blgrad0 = blgrad0 - dthb
        
        adjustb!(blgrad0, blgradi_ref, a, bw, cw, dw, xmaj, plrad, ishape)
        blgradi = blgradi_ref[]

        for i in 1:mth2
            the0 = (i - 1) * dth
            thbulg = (the0 > 0.5 * pye && the0 < 1.5 * pye) ? blgrado : blgradi
            cost2b = cos(2.0 * thbulg)
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 - aw * ferm
            the2 = brad * sin(the)
            cost2 = cos(2.0 * the2)
            fermb = 1.0 / (exp((cost2b - cost2) / tbulg) + 1.0)
            bulge = abulg * fermb
            xwal1[i] = cw + rho * cos(the2 + dw * sin(the2)) + bulge
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=34 Shell independent of plasma. Gap on the outer side.
    if ishape == 34
        a0 = a + aw
        brad = b * pye / 180.0
        wcentr = cw
        for i in 1:mth2
            the0 = (i - 1) * dth
            the = the0
            cost = cos(the)
            ferm = +1.0 - 2.0 / (exp(cost / tw) + 1.0)
            rho = a0 + aw * ferm
            the2 = brad * sin(the)
            xwal1[i] = cw - rho * cos(the2 - dw * sin(the2))
            zwal1[i] = -bw * rho * sin(the2)
        end
    end

    # ishape=41 Arbitrary wall generated by spline data from wall_geo.io
    if ishape == 41

        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            # Allocate arrays based on npots0, as per Fortran's dynamic allocation
            thetatmp = zeros(Float64, npots0)
            xwaltmp = zeros(Float64, npots0)
            zwaltmp = zeros(Float64, npots0)
            rioptmp = zeros(Int, 2)
            xpptmp = zeros(Float64, npots0)
            ww1tmp = zeros(Float64, npots0)
            ww2tmp = zeros(Float64, npots0)
            ww3tmp = zeros(Float64, npots0)
            tabtmp = zeros(Float64, 3)

            for i in 1:npots0
                line = split(readline(io))
                thetatmp[i] = parse(Float64, line[1])
                xwaltmp[i] = parse(Float64, line[2]) - wcentr
                zwaltmp[i] = parse(Float64, line[3])
            end
        end # `do io` block automatically closes the file

        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, xwaltmp, xpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2(npots0,thetatmp,xwaltmp,xpptmp,1,the0,tabtmp)
            xwal1[i] = tabtmp[1]*a + wcentr
        end

        rioptmp[1] = 4
        rioptmp[2] = 4
        spl1d1!(npots0, thetatmp, xwaltmp, zpptmp, rioptmp, 1, ww1tmp, ww2tmp, ww3tmp)

        for i in 1:mth1
            the0 = (i - 1) * dth
            spl1d2(npots0,thetatmp,zwaltmp,xpptmp,1,the0,tabtmp)
            zwal1[i] =tabtmp[1]*a
        end

        xwal1[1] = xwal1[mth1]
        zwal1[1] = zwal1[mth1]
        xwal1[mth2] = xwal1[2]
        zwal1[mth2] = zwal1[2]
    end

    # ishape=42 Arbitrary wall generated by position data
    if ishape == 42
        open("wall_geo.in", "r") do io
            npots0 = parse(Int, readline(io))
            wcentr = parse(Float64, readline(io))
            readline(io) # Skip the next line

            if npots0 != mth + 2
                @error "ERROR: Number of points in wall_geo.in must be equal to mth+2."
                error("Wall geometry error") # Stop execution
            end

            thetatmp_dummy = zeros(Float64, npots0)
            for i in 1:npots0
                line = split(readline(io))
                thetatmp_dummy[i] = parse(Float64, line[1]) # Read but not used
                xwal1[i] = parse(Float64, line[2])
                zwal1[i] = parse(Float64, line[3])
            end
        end

        if xwal1[mth1] != xwal1[1] || zwal1[mth1] != zwal1[1]
            @error "ERROR: First point in wall_geo.in must be equal to 2nd last point."
            error("Wall geometry error")
        end
        if xwal1[mth2] != xwal1[2] || zwal1[mth2] != zwal1[2]
            @error "ERROR: Last point in wall_geo.in must be equal to 2nd point."
            error("Wall geometry error")
        end
    end

    xmx = xma + xshift

    # --- Cleanup ---
    @label cleanup # Target for the `goto` from ishape=-10
    if leqarcw == 1
        eqarcw!( xwal1,zwal1, xpp,zpp, ww1,ww2,ww3, mth1 )
        for i in 1:mth1
            xwal1[i] = xpp_out[i]
            zwal1[i] = zpp_out[i]
        end
    end

    if iplt <= 0
        xmx = xmaj
        zma = 0.0
        iplt = 1
    end


    aw = awsave # restore value of aw
    bw = bwsave # restore value of bw

end

wwall!

In [13]:
# If you are using the Interpolations.jl package in Julia, you need to declare 'using' first.
# using Interpolations # No need to redeclare if already done.

# --- 1. Set up necessary global variables (must be defined outside the wwall! function) ---
# These variables should match the global or module variables in your actual code.
# Here, only a few key variables are listed for example purposes.
nths = 100 # Number of theta angle points to construct the wall (adjustable)
ishape = 8 # Select DIII-D wall shape.
aw = 0.0
bw = 0.0
a = 3.0
b = 2.0
cw = 1.0
dw = 0.0
tw = 0.0
abulg = 0.0
bbulg = 0.0
tbulg = 0.0
farwal = false
leqarcw = 0
xinf = zeros(Float64, nths)
zinf = zeros(Float64, nths)
xma = 0.0
zma = 0.0
xmx = 0.0
zmx = 0.0
iplt = 0
xmin
outmod = 0 # Initialized as a global variable
iotty = 0  # Initialized as a global variable

# --- 2. Prepare arrays to store wall coordinates ---
# `xwal1` and `zwal1` are arrays that will be populated by the `wwall!` function.
# Since `wwall!` uses indices like `mth1`, `mth2`, etc.,
# the array size should be large enough, matching Fortran's `DIMENSION` declaration.
# Generally, `mth2` might be `nths + 2`, so initialize safely with `nths + 2`.
# (If in Fortran, `mth1 = nths` and `mth2 = nths`, then `nths` size would suffice.
#  It is most accurate to check the array size declarations in the original Fortran code.)
x_wall_coordinates = zeros(Float64, nths + 2)
z_wall_coordinates = zeros(Float64, nths + 2)

# --- 3. Call the wwall! function ---
# The `nqnqnq` argument's specific use is not clear from the provided snippet,
# but a dummy value like 0 is passed to match the Fortran function signature.
wwall!(0, x_wall_coordinates, z_wall_coordinates)

# --- 4. Verify results ---
println("--- DIII-D Wall Coordinates ---")
println("X-coordinates (R):")
for i in 1:min(10, length(x_wall_coordinates)) # Example: Print only the first 10 coordinates
    println("  Point $(i): $(x_wall_coordinates[i])")
end

println("\nZ-coordinates:")
for i in 1:min(10, length(z_wall_coordinates)) # Example: Print only the first 10 coordinates
    println("  Point $(i): $(z_wall_coordinates[i])")
end

# Use the following lines if you want to print all coordinates:
# println("\nAll X-coordinates: $(x_wall_coordinates)")
# println("All Z-coordinates: $(x_wall_coordinates)") # Corrected: should be z_wall_coordinates

ier in d3dwall =   0
--- DIII-D Wall Coordinates ---
X-coordinates (R):
  Point 1: 2.4466479142260074
  Point 2: 2.4439749936354946
  Point 3: 2.4404910482865985
  Point 4: 2.4415666781154317
  Point 5: 2.444202169966597
  Point 6: 2.443383485985041
  Point 7: 2.4403590650694964
  Point 8: 2.4368785275161007
  Point 9: 2.4281844690276424
  Point 10: 2.410034176048802

Z-coordinates:
  Point 1: 0.0
  Point 2: -0.05278982558852149
  Point 3: -0.1021974422343911
  Point 4: -0.15274916889184947
  Point 5: -0.2083920347925847
  Point 6: -0.2650544853077749
  Point 7: -0.3194084738126985
  Point 8: -0.37548553074489766
  Point 9: -0.4354845760466677
  Point 10: -0.4941166443029736
